In [1]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import re
import numpy as np

In [ ]:
rate = pd.read_excel(r"path\to\rate_card.xlsx", engine='openpyxl')
current_dir = os.getcwd()
group_data = pd.DataFrame()

# UPS warehouse names are determined by account numbers in the table.
ups_location_df = pd.read_excel(r"path\to\account_number.xlsx", sheet_name="Account")
account_to_location_dict = dict(zip(ups_location_df['Account Number'], ups_location_df['Warehouse Location']))

# FedEx-O warehouse names are determined by shipping zip codes.
orsd_location_df = pd.read_excel(r"path\to\warehouse_zip.xlsx", sheet_name="邮编")
zip_to_location_dict = dict(zip(orsd_location_df['发货邮编'], orsd_location_df['Warehouse Location']))

In [ ]:
def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx') or file.endswith('.xls'):
            files.append(os.path.join(folder_path, file))
    return files

def count_sheets_in_file(file_path):
    try:
        excel_file = pd.ExcelFile(file_path)
        sheet_count = len(excel_file.sheet_names)
        return sheet_count
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return 0

        
def remove_special_characters(text):
    # Check if the input is not a string
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()
    text = re.sub(r"[\n\r\t]", "", text)
    return re.sub(r'[^\u4e00-\u9fa5\w]+', '', text)

def convert_waybill_number(value):
    # Must have '\t' to successfully convert to str without scientific notation.
    if isinstance(value, (int, float)):
        return str(int(value)) + '\t'
    else:
        return str(value) + '\t'

    
def read_and_append(file_path):
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        df.fillna(0, inplace=True)
        df['Waybill Number'] = df['Waybill Number'].apply(convert_waybill_number)
        return df
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame()


def calculate_fee(df):
    conditions = (df['计费重量(lbs)'] <= 150) & (df['Zone'] > 0)

    fee = np.where(conditions, rate.values[df['计费重量(lbs)']-1, df['Zone'] - 1], np.nan)
    
    return fee

def merge_files_to_sheet(current_dir, folder_path):
    # Each folder contains invoices for a month from a specific carrier.
    # Merge files within a folder to create a summary sheet named after the folder.
    folder_name = os.path.basename(folder_path)
    summary_file = os.path.join(current_dir, f"{folder_name}.csv")

    files = get_files_in_folder(folder_path)

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(read_and_append, file_path) for file_path in files]

        merged_df = pd.DataFrame()

        for future in futures:
            result = future.result()
            if not result.empty:
                merged_df = pd.concat([merged_df, result], ignore_index=False)
    
    if folder_name == 'FedexO':
    # For 'FedexO':
    # 1. Round up '计费重量(lbs)' and convert to integer.
    # 2. Convert 'Zone' to integer.
    # 3. Calculate '基础运费' and '应收金额'.
    # 4. Fill missing '基础运费' and '应收金额' values.
    # 5. Map '发货邮编' to 'Warehouse Location'.

        merged_df['计费重量(lbs)'] = np.ceil(merged_df['计费重量(lbs)']).astype(int)
        merged_df['Zone'] = merged_df['Zone'].astype(int)       
        merged_df['基础运费'] = calculate_fee(merged_df).astype(float)
        merged_df['应收金额'] = merged_df['基础运费'] + merged_df['总费用'].astype(float) - merged_df['运费'].astype(float)
        merged_df['基础运费'].fillna(merged_df['运费'], inplace=True)
        merged_df['应收金额'].fillna(merged_df['总费用'], inplace=True)
        merged_df['Warehouse Location'] = merged_df['发货邮编'].map(zip_to_location_dict)
    

    if folder_name == 'UPS':
    # For 'UPS':
    # 1. Extract last 6 characters of 'Account Number'.
    # 2. Map 'Account Number' to 'Warehouse Location'.
    # 3. Save rows without 'Warehouse Location' to 'UPS非自营仓.csv'.
    # 4. Drop rows without 'Warehouse Location'.

        merged_df['Account Number'] = merged_df['Account Number'].astype(str).str[-6:]

        merged_df['Warehouse Location'] = merged_df['Account Number'].map(account_to_location_dict)
        merged_df_without_location = merged_df[merged_df['Warehouse Location'].isna()]
        non_self_storage_file = os.path.join(current_dir, "UPS非自营仓.csv")
        merged_df_without_location.to_csv(non_self_storage_file, index=False, encoding='utf-8-sig')
        merged_df = merged_df.dropna(subset=['Warehouse Location'])


    merged_df.to_csv(summary_file, index=False, encoding='utf-8-sig')
     
    group_df = calculate_revenue_and_cost(merged_df, folder_name)

    return group_df
    
        


def calculate_revenue_and_cost(df, carrier_name):
     # Determine the appropriate revenue and cost columns based on the carrier name
    if carrier_name == 'epost':
        revenue_column = 'JD Total Amount'
        cost_column = 'total_amt'
    elif carrier_name == 'DHLeco':
        revenue_column = 'JD Total Amount'
        cost_column = 'Base Amount'
    elif carrier_name == 'DHLexp':
        revenue_column = 'JD Total Amount'
        cost_column = 'Total Charge'
    elif carrier_name == 'USPS':
        revenue_column = 'JD Total Amount'
        cost_column = 'tier_price'
    elif carrier_name == 'FedexL':
        revenue_column = 'JD Total Amount'
        cost_column = 'Total Charge'
    elif carrier_name == 'FedexA':
        revenue_column = 'JD Total Amount'
        cost_column = 'Total Amount'
    elif carrier_name == 'FedexO':
        revenue_column = '应收金额'
        cost_column = '总费用'
    elif carrier_name == 'ACI':
        revenue_column = 'JD Total'
        cost_column = 'final invoice amt'
    elif carrier_name == 'Uniuni':
        revenue_column = 'JD Total'
        cost_column = 'Shipping Fee'
    elif carrier_name == 'UPS':
        revenue_column = 'Net Amount'
        cost_column = 'Net Amount'
    else:
        print("error")

    df['Cost'] = pd.to_numeric(df[cost_column], errors='coerce').fillna(0)
    df['Revenue'] = pd.to_numeric(df[revenue_column], errors='coerce').fillna(0)

    warehouse_mapping = {
    "ca1": "美国洛杉矶1号仓",
    "ca2": "美国洛杉矶2号仓",
    "ca3": "美国洛杉矶3号仓",
    "ga1": "美国亚特兰大1号仓",
    "nj1": "美国新泽西1号仓",
    "nj2": "美国新泽西2号仓"
}
    # Group data by 'Customer Name' and 'Warehouse Location' if the column exists
    if 'Warehouse Location' in df.columns:
        group_df = df.groupby(['Customer Name','Warehouse Location'])[['Revenue','Cost']].sum().reset_index()
        warehouse_name = group_df['Warehouse Location'].map(warehouse_mapping)

        group_df.insert(0, '仓中文名称', warehouse_name)
    else:
        group_df = df.groupby(['Customer Name'])[['Revenue','Cost']].sum().reset_index()

    group_df['delivery'] = carrier_name

    
    return group_df



def split_files_by_customer_name(current_dir, folder_path):
    folder_name = os.path.basename(folder_path)
    merged_file = os.path.join(current_dir, f"{folder_name}.csv")
    print(merged_file)

    df = pd.read_csv(merged_file)
    df['Waybill Number'] = df['Waybill Number'].apply(convert_waybill_number)
    for customer_name in df['Customer Name'].unique():
        if pd.isna(customer_name):
            print(df[df['Customer Name'].isna()])
            df = df.dropna(subset=['Customer Name'])
        customer_name_cleaned = remove_special_characters(customer_name)
        customer_folder = os.path.join(r"path\to\output_folder", customer_name_cleaned)
        if not os.path.exists(customer_folder):
            os.makedirs(customer_folder)
        
        customer_file = f"{customer_name_cleaned}_{folder_name}.csv"
        customer_path = os.path.join(customer_folder, customer_file)
        customer_df = df[df['Customer Name'] == customer_name]
        customer_df.to_csv(customer_path, index=False, encoding='utf-8-sig')

In [ ]:
folders = [folder for folder in os.listdir() if os.path.isdir(folder)]


for folder in folders:
    folder_path = os.path.join(os.getcwd(), folder)
    print(folder)
    # files = get_files_in_folder(folder_path)
    # for file in files:
    #     sheet_count = count_sheets_in_file(file)
    #     print(file)
    #     print(sheet_count)   

    group_df = merge_files_to_sheet(current_dir, folder_path)
    group_data = pd.concat([group_data, group_df], ignore_index=True)
    split_files_by_customer_name(current_dir, folder_path)

group_data.to_excel("尾程收入成本.xlsx", index=False, engine='openpyxl')

In [ ]:
# Only running a single folder

# folder_path = (r"path\to\single_folder")
# # files = get_files_in_folder(folder_path)）
# # for file in files:
# #     print(file)
# #     print(count_sheets_in_file(file))
# merge_files_to_sheet(current_dir, folder_path)
# split_files_by_customer_name(current_dir, folder_path)